# PyTorch Quick start

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Load Data

In [2]:
from datasets import load_dataset, Audio
fashion_mnist = load_dataset("fashion_mnist")

Found cached dataset fashion_mnist (/root/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
fashion_mnist

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [4]:
fashion_mnist['train'][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [5]:
from torch.utils.data import DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_set = fashion_mnist['train'].with_format("torch", device=device)
test_set = fashion_mnist['test'].with_format("torch", device=device)

# add a dimension of size one, so that the shape of the tensor is (batch_size, 1, 28, 28) instead of (batch_size, 28, 28).
train_set = train_set.map(lambda x: {'image': x['image'].unsqueeze(0).to(torch.float32), 'label': x['label']})
test_set = test_set.map(lambda x: {'image': x['image'].unsqueeze(0).to(torch.float32), 'label': x['label']})



# Create data loaders.
batch_size = 64
train_dataloader = DataLoader(train_set, batch_size=batch_size)
test_dataloader = DataLoader(test_set, batch_size=batch_size)

for batch in test_dataloader:
    X = batch['image']
    y = batch['label']
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Loading cached processed dataset at /root/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48/cache-e2843ed7a653f764.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/0a671f063342996f19779d38c0ab4abef9c64f757b35af8134b331c294d7ba48/cache-80eaa5fc60a81463.arrow


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
test_set[0]

{'image': tensor([[[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.],
          [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
             0.,   0.,   0., 

## Create Model

In [7]:
from typing import Optional
from torch import Tensor
from transformers import PreTrainedModel, PretrainedConfig
from transformers.modeling_outputs import ImageClassifierOutputWithNoAttention
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

class NeuralNetworkConfig(PretrainedConfig):
    model_type = "neural_network"
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Define model
class NNForImageCalssification(PreTrainedModel):
    config_class = NeuralNetworkConfig
    def __init__(self, config):
        super().__init__(config)
        self.model = NeuralNetwork()

    def forward(
        self,
        pixel_values: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> ImageClassifierOutputWithNoAttention:
        
        logits = self.model(pixel_values)
        if labels is not None:
            loss = torch.nn.cross_entropy(logits, labels)
            return {'loss': loss, 'logits': logits}
        return {'logits': logits}
       

model = NNForImageCalssification(NeuralNetworkConfig())
model.to(device)
model

NNForImageCalssification(
  (model): NeuralNetwork(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear_relu_stack): Sequential(
      (0): Linear(in_features=784, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=10, bias=True)
    )
  )
)

In [8]:
input = torch.randn(1, 1, 28, 28).to(device)
output = model(input)
output

{'logits': tensor([[-1.6463e-01,  7.5585e-02,  1.2435e-01,  1.6102e-01,  6.3234e-02,
          -5.5278e-02,  2.9660e-02,  2.8994e-02,  4.7613e-05, -6.0434e-02]],
        device='cuda:0', grad_fn=<AddmmBackward0>)}

## Train Model

In [9]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="p",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=fashion_mnist["train"],
    eval_dataset=fashion_mnist["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

/root/.cache/pypoetry/virtualenvs/e2e-audioclassfication-xS3fZVNL-py3.8/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AttributeError: Caught AttributeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/root/.cache/pypoetry/virtualenvs/e2e-audioclassfication-xS3fZVNL-py3.8/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/root/.cache/pypoetry/virtualenvs/e2e-audioclassfication-xS3fZVNL-py3.8/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_1041327/2552776656.py", line 45, in forward
    logits = self.model(pixel_values)
  File "/root/.cache/pypoetry/virtualenvs/e2e-audioclassfication-xS3fZVNL-py3.8/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_1041327/2552776656.py", line 26, in forward
    x = self.flatten(x)
  File "/root/.cache/pypoetry/virtualenvs/e2e-audioclassfication-xS3fZVNL-py3.8/lib/

In [56]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [63]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_no, batch in enumerate(dataloader):
        X = batch['image']
        y = batch['label']

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_no % 100 == 0:
            loss, current = loss.item(), (batch_no + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [58]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            X = batch['image']
            y = batch['label']
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [64]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 77.949959  [   64/60000]
loss: 0.687569  [ 6464/60000]
loss: 0.459620  [12864/60000]
loss: 0.519396  [19264/60000]
loss: 0.665099  [25664/60000]
loss: 0.577330  [32064/60000]
loss: 0.485165  [38464/60000]
loss: 0.554667  [44864/60000]
loss: 0.558046  [51264/60000]
loss: 0.538543  [57664/60000]
Test Error: 
 Accuracy: 82.0%, Avg loss: 0.497685 

Epoch 2
-------------------------------
loss: 0.357045  [   64/60000]
loss: 0.414865  [ 6464/60000]
loss: 0.265716  [12864/60000]
loss: 0.374755  [19264/60000]
loss: 0.390263  [25664/60000]
loss: 0.486630  [32064/60000]
loss: 0.372526  [38464/60000]
loss: 0.460857  [44864/60000]
loss: 0.450088  [51264/60000]
loss: 0.465171  [57664/60000]
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.429946 

Epoch 3
-------------------------------
loss: 0.275551  [   64/60000]
loss: 0.355248  [ 6464/60000]
loss: 0.241884  [12864/60000]
loss: 0.359476  [19264/60000]
loss: 0.301658  [25664/60000]
loss: 0.443956  [32064/60

## Saving / Loading Models

In [65]:
# print current path
import os
# change to /workspace
os.chdir('/workspace')
print(os.getcwd())

/workspace


In [66]:
path = "outputs/pytorch-test/model-2.pth"
torch.save(model.state_dict(), path)
print("Saved PyTorch Model State to" + path)


Saved PyTorch Model State tooutputs/pytorch-test/model-2.pth


In [67]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [71]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_set[122]['image'], test_set[122]['label']
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')


Predicted: "Ankle boot", Actual: "Ankle boot"
